# Introduction

In this notebook I scrape image data from Google, Yandex, and Flickr on Google Chrome. The goal is to add image data to the [Flowers Recognition](https://www.kaggle.com/alxmamaev/flowers-recognition) data set available on Kaggle with new species of flowers. So far I have used this code to add images of the following flower species to the [Flowers Recognition](https://www.kaggle.com/alxmamaev/flowers-recognition) data set:

- aster
- orchid

## Load all packages

In [1]:
from PIL import Image
import os
from selenium import webdriver
import time
import io
import requests
import hashlib

## Scraping from Google Chrome

The bit of code below functions to scrape images from Google. This method is based on this post which gives info on how to scrape image data from Google: [https://medium.com/p/a96feda8af2d](https://medium.com/p/a96feda8af2d)

In [2]:
search_term = 'iris'

In [3]:
DRIVER_PATH = './chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [4]:
wd.get('https://google.com')

In [5]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [6]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG")
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [7]:
def search_and_download(search_term:str,driver_path:str,target_path='./flowers',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In the code below we may have to adjust number of images we can pull from Google. If it finds less than the number of images indicated it will return an error.

In [8]:
img_path = 'flowers/' + search_term + '/'

if search_term not in os.listdir('flowers/.'): 
    os.mkdir(img_path)

search_and_download(search_term=search_term,driver_path=DRIVER_PATH,number_images=130)

Found: 100 search results. Extracting links from 0:100
Found: 131 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSjVTmYugi6JeK3QDz-koR6m86Qb0VKKg3bWQ&usqp=CAU - as ./flowers/iris/a42d1c7c13.jpg
SUCCESS - saved https://www.mygarden.com/sites/default/files/wieseniris-iris-sibirica-1-472194-blp-msg.jpg - as ./flowers/iris/f19b9540a4.jpg
SUCCESS - saved https://images.fineartamerica.com/images/artworkimages/mediumlarge/3/wild-iris-isabela-and-skender-cocoli.jpg - as ./flowers/iris/b84c36cf72.jpg
SUCCESS - saved https://www.thespruce.com/thmb/BraXhPcSU3pKHYLfVo1eQid4uOg=/4200x2362/smart/filters:no_upscale()/how-to-grow-and-care-for-japanese-iris-5077439-hero-bbbd19a517fa4c01a6ec5d7b70e67969.JPG - as ./flowers/iris/910a3eb62a.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQUimKcVtiKuGhaq2S2xn2qxRucGgib46V1bw&usqp=CAU - as ./flowers/iris/cd56836f60.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRtG

SUCCESS - saved https://www.thespruce.com/thmb/evxObrHgLUPb22qDX1ZVA_q2gB0=/3761x2528/filters:fill(auto,1)/taxonomy-of-reticulated-iris-2132119_06-3df4ef12453a482eb1093b4de63b6c97.jpg - as ./flowers/iris/5bcb27c5e1.jpg
SUCCESS - saved https://i.ytimg.com/vi/XI3iMgfIXlY/maxresdefault.jpg - as ./flowers/iris/4fec903354.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQBrM3pmUbkbaivDwP-GCzWFrOoInDf2l3M5w&usqp=CAU - as ./flowers/iris/27cb621118.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQH05y5aFw5itiVJGIgw9B6h-BUh_KlDv_3sw&usqp=CAU - as ./flowers/iris/552fa618b0.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRpsfjwQJLybp7JEcPgEiVip4wtBE36MHaQeA&usqp=CAU - as ./flowers/iris/78389b8716.jpg
SUCCESS - saved https://cdn.shoplightspeed.com/shops/601459/files/151385/iris-versicolor-iris-blue-flag-1.jpg - as ./flowers/iris/bef4ae7602.jpg
SUCCESS - saved https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2F

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQghlFRNhY53vaGn_9agjDJ0x-xVo9R8Qehrg&usqp=CAU - as ./flowers/iris/2eae996944.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSEcDB1qbYHOcj4ZaxQuIP_6XMjEwbQ4exPMw&usqp=CAU - as ./flowers/iris/57cf467aed.jpg
SUCCESS - saved https://www.bluestoneperennials.com/img/IRCB/650/IRCB_0_Iris_Caesars_Brother.1491334114.jpg - as ./flowers/iris/f38dfad9ed.jpg
SUCCESS - saved https://cdn11.bigcommerce.com/s-ih8o56kgor/images/stencil/1280x1280/products/4525/7247/siberian-iris-iris-sibirica__00262.1564590282.jpg?c=2 - as ./flowers/iris/1e2ccedf81.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRWFgYg6G0xL8LTxFELhQnWqAf1ciTpAn6ujg&usqp=CAU - as ./flowers/iris/3ac88d8590.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRu0rHEEB5ePup3j0Ln8KFkfybotn__5LQv1w&usqp=CAU - as ./flowers/iris/707e60df70.jpg
SUCCESS - saved https://www.pickupflowers.com/images/assets/i

## Scraping images on Flickr

The two lines of code search Flickr for the number of images specified after the search term. See the following repository to download, setup, and get 'flickrGetUrl.py' and 'get_images.py' to run: [https://github.com/ultralytics/flickr_scraper](https://github.com/ultralytics/flickr_scraper). The versions of that code that are run here are slightly modified for easier use in this repository.

In [9]:
run flickrGetUrl.py iris 500

Fetching url for image number 1
Fetching url for image number 2
Fetching url for image number 3
Fetching url for image number 4
Fetching url for image number 5
Fetching url for image number 6
Fetching url for image number 7
Fetching url for image number 8
Fetching url for image number 9
Fetching url for image number 10
Fetching url for image number 11
Fetching url for image number 12
Fetching url for image number 13
Fetching url for image number 14
Fetching url for image number 15
Fetching url for image number 16
Fetching url for image number 17
Fetching url for image number 18
Fetching url for image number 19
Fetching url for image number 20
Fetching url for image number 21
Fetching url for image number 22
Fetching url for image number 23
Fetching url for image number 24
Fetching url for image number 25
Fetching url for image number 26
Fetching url for image number 27
Fetching url for image number 28
Fetching url for image number 29
Fetching url for image number 30
Fetching url for im

Fetching url for image number 251
Fetching url for image number 252
Fetching url for image number 253
Fetching url for image number 254
Fetching url for image number 255
Fetching url for image number 256
Fetching url for image number 257
Fetching url for image number 258
Fetching url for image number 259
Fetching url for image number 260
Fetching url for image number 261
Fetching url for image number 262
Fetching url for image number 263
Fetching url for image number 264
Fetching url for image number 265
Fetching url for image number 266
Fetching url for image number 267
Fetching url for image number 268
Fetching url for image number 269
Fetching url for image number 270
Fetching url for image number 271
Fetching url for image number 272
Fetching url for image number 273
Fetching url for image number 274
Fetching url for image number 275
Fetching url for image number 276
Fetching url for image number 277
Fetching url for image number 278
Fetching url for image number 279
Fetching url f

Done fetching urls, fetched 500 urls out of 500
Writing out the urls in the current directory
Done!!!


In [10]:
run get_images.py image_urls.csv iris

Starting download 1 of  341
Done downloading 1 of 341
Starting download 2 of  341
Done downloading 2 of 341
Starting download 3 of  341
Done downloading 3 of 341
Starting download 4 of  341
Done downloading 4 of 341
Starting download 5 of  341
Done downloading 5 of 341
Starting download 6 of  341
Done downloading 6 of 341
Starting download 7 of  341
Done downloading 7 of 341
Starting download 8 of  341
Done downloading 8 of 341
Starting download 9 of  341
Done downloading 9 of 341
Starting download 10 of  341
Done downloading 10 of 341
Starting download 11 of  341
Done downloading 11 of 341
Starting download 12 of  341
Done downloading 12 of 341
Starting download 13 of  341
Done downloading 13 of 341
Starting download 14 of  341
Done downloading 14 of 341
Starting download 15 of  341
Done downloading 15 of 341
Starting download 16 of  341
Done downloading 16 of 341
Starting download 17 of  341
Done downloading 17 of 341
Starting download 18 of  341
Done downloading 18 of 341
Starting d

Done downloading 146 of 341
Starting download 147 of  341
Done downloading 147 of 341
Starting download 148 of  341
Done downloading 148 of 341
Starting download 149 of  341
Done downloading 149 of 341
Starting download 150 of  341
Done downloading 150 of 341
Starting download 151 of  341
Done downloading 151 of 341
Starting download 152 of  341
Done downloading 152 of 341
Starting download 153 of  341
Done downloading 153 of 341
Starting download 154 of  341
Done downloading 154 of 341
Starting download 155 of  341
Done downloading 155 of 341
Starting download 156 of  341
Done downloading 156 of 341
Starting download 157 of  341
Done downloading 157 of 341
Starting download 158 of  341
Done downloading 158 of 341
Starting download 159 of  341
Done downloading 159 of 341
Starting download 160 of  341
Done downloading 160 of 341
Starting download 161 of  341
Done downloading 161 of 341
Starting download 162 of  341
Done downloading 162 of 341
Starting download 163 of  341
Done downloadi

Done downloading 288 of 341
Starting download 289 of  341
Done downloading 289 of 341
Starting download 290 of  341
Done downloading 290 of 341
Starting download 291 of  341
Done downloading 291 of 341
Starting download 292 of  341
Done downloading 292 of 341
Starting download 293 of  341
Done downloading 293 of 341
Starting download 294 of  341
Done downloading 294 of 341
Starting download 295 of  341
Done downloading 295 of 341
Starting download 296 of  341
Done downloading 296 of 341
Starting download 297 of  341
Done downloading 297 of 341
Starting download 298 of  341
Done downloading 298 of 341
Starting download 299 of  341
Done downloading 299 of 341
Starting download 300 of  341
Done downloading 300 of 341
Starting download 301 of  341
Done downloading 301 of 341
Starting download 302 of  341
Done downloading 302 of 341
Starting download 303 of  341
Done downloading 303 of 341
Starting download 304 of  341
Done downloading 304 of 341
Starting download 305 of  341
Done downloadi

## Scraping images on Yandex

The next line of code searches Yandex for images of the search term and puts them in a temporary folder named 'downloads' which I then delete after moving the images out of there. Note that these are run outside of Python (hence the ! before each command).

In [11]:
! yandex-images-download Chrome --keywords "iris" --limit 500 

Output directory is set to "downloads/"
Limit of images is set to 500
  Found 50 pages of iris.
  Scrapping page 1/17...
    Downloaded the image. ==> downloads/iris/http-yakimenko-iris-net-ua-uploads-posts-2016-06.jpg
    Downloaded the image. ==> downloads/iris/irises-2095663_1280.jpg
    fail: https://mykaleidoscope.ru/uploads/posts/2021-09/1632713817_42-mykaleidoscope-ru-p-iris-sladost-krasivo-foto-46.jpg error: img_url response is not ok. response: <Response [404]>.
    Downloaded the image. ==> downloads/iris/Irises_30.jpg
    Downloaded the image. ==> downloads/iris/9d84944f14ed1ed6fd5d765ab2135dd5.jpg
    fail: https://cvet-dom.ru/wp-content/uploads/2017/07/Iris.jpg error: img_url response is not ok. response: <Response [403]>.
    Downloaded the image. ==> downloads/iris/26ac6560e44f91c4720358d05f5ebda7.jpg
    Downloaded the image. ==> downloads/iris/kak-obrezat-irisy-posle-cveteniya.jpg
    Downloaded the image. ==> downloads/iris/Irises_39.jpg
    Downloaded the image. ==> 

    Downloaded the image. ==> downloads/iris/scale_1200 (6).jpg
    Downloaded the image. ==> downloads/iris/scale_1200 (7).jpg
    Downloaded the image. ==> downloads/iris/Irises_Closeup_Violet_353034.jpg
    Downloaded the image. ==> downloads/iris/scale_1200 (8).jpg
    Downloaded the image. ==> downloads/iris/scale_1200 (9).jpg
    Downloaded the image. ==> downloads/iris/flower-2517562_1280.jpg
    Downloaded the image. ==> downloads/iris/post_817_27325.jpg
    Downloaded the image. ==> downloads/iris/2048x1360_811927_[www.ArtFile.ru].jpg
    Downloaded the image. ==> downloads/iris/MOONLIT_WATER-414-B.jpg
    Downloaded the image. ==> downloads/iris/0f90a28aa43f376238eab1a168d9dab3.jpe
    Downloaded the image. ==> downloads/iris/622018ce02070920a8796cad594d897e.jpg
    Downloaded the image. ==> downloads/iris/picture.jpg
    Downloaded the image. ==> downloads/iris/1613153274_15-p-irisi-na-zheltom-fone-22.jpg
    Downloaded the image. ==> downloads/iris/vin-3-3.jpg
    Downloade

    Downloaded the image. ==> downloads/iris/irisy-04.jpg
    Downloaded the image. ==> downloads/iris/purple-iris-967845_1280.jpg
    Downloaded the image. ==> downloads/iris/plant-flower-purple-petal-high-flora-iris-eye-hire.jpg
    Downloaded the image. ==> downloads/iris/iris-791894_1280.jpg
    Downloaded the image. ==> downloads/iris/iris-flower-2327697_1280.jpg
    Downloaded the image. ==> downloads/iris/0a1a8e9dfec148f13438f5722f9362e8.jpeg
    Downloaded the image. ==> downloads/iris/1613153304_116-p-irisi-na-zheltom-fone-167.jpg
  Scrapping page 8/17...
    Downloaded the image. ==> downloads/iris/69234-iris-borodatyy-pati-dress.jpg
    Downloaded the image. ==> downloads/iris/iris-2691787_1280.jpg
    Downloaded the image. ==> downloads/iris/iris-1362576_1280.jpg
    Downloaded the image. ==> downloads/iris/picture.jpg (1).jpg
    Downloaded the image. ==> downloads/iris/2-7.jpg
    Downloaded the image. ==> downloads/iris/s1200-2.jpg
    Downloaded the image. ==> downloads

    Downloaded the image. ==> downloads/iris/iris-kasatik-fioletovyy-makro.jpg
    Downloaded the image. ==> downloads/iris/dc8c8b29f28c6f2c7017327d5e75882e.jpg
    Downloaded the image. ==> downloads/iris/204abb6aa27f44a8b42b5727482dd628.max-1200x800.jpg
    Downloaded the image. ==> downloads/iris/4cdd66e272723342154cdee5ce273c8f.jpg
    Downloaded the image. ==> downloads/iris/scale_1200 (15).jpg
    fail: https://upload.wikimedia.org/wikipedia/commons/c/cb/Iris_croatica_Botanicki_vrt_1_080509.jpg error: img_url response is not ok. response: <Response [403]>.
    Downloaded the image. ==> downloads/iris/Irises_27.jpg
    Downloaded the image. ==> downloads/iris/nature-plant-flower-purple-spring-macro-botany-gar.jpg
    Downloaded the image. ==> downloads/iris/59704072b278e_iris.jpg
    Downloaded the image. ==> downloads/iris/iris-1467955_1280.jpg
    Downloaded the image. ==> downloads/iris/borodatyj-iris.jpg
    Downloaded the image. ==> downloads/iris/d657d7f5ade93d6665147d58f9e4

    Downloaded the image. ==> downloads/iris/iris55.jpg
    Downloaded the image. ==> downloads/iris/2471646.jpg
    Downloaded the image. ==> downloads/iris/b8871085b383ce3bd8bf8b1215fcf770--plants-for-garde.jpg
    Downloaded the image. ==> downloads/iris/97cebbeac3482284bc745b31a58e5e7b.jpg
    Downloaded the image. ==> downloads/iris/--------------------21-1200x900.jpg
    Downloaded the image. ==> downloads/iris/d4c3401954ac8faf4226cf8f86a7f556.png
    Downloaded the image. ==> downloads/iris/0_82b79_7c76b1b6_-2-XL.jpg
    Downloaded the image. ==> downloads/iris/cbb136cba84b422355c1233ac789832d--bright-spring-si.jpg
  Scrapping page 15/17...
    Downloaded the image. ==> downloads/iris/purple-iris-2509690_1280.jpg
    Downloaded the image. ==> downloads/iris/iris-4587342_1280.jpg
    fail: https://mocah.org/uploads/posts/1014912-flowers-plants-macro-Iris-flower-plant-eye-flora-wildflower-botany-land-plant-flowering-plant-macro-photography-iris-versicolor-iris-family.jpg error: im

### Do some cleanup
Now I just want to move all files to the corresponding flowers directory and remove all duplicate files. Duplicates are downloaded with a space and number in the name (i.e., 'file (1).jpg' would be a duplicate of 'file.jpg').

In [12]:
! mv downloads/iris/* flowers/iris/.

In [13]:
! rm -rf downloads

In [14]:
! rm flowers/iris/*\ *